This is a Jupyter Notebook. It allows me to use markdown(text) to explain concepts, and add actual code that can be run within this browser. In order to run a section of code, simply select the cell the code is in, and press Ctrl+Enter, or click on the run symbol in the toolbar. This notebook intends to show some basic Excel manipulations using Python. I highly suggest copying this code for your own use, but feel free to observe how it will run and some sample data included in this notebook. First things first, let's import the package that you will become very familiar with when handling data... pandas.

In [ ]:
import pandas as pd

The pandas package is a commonly used library of tools used to manipulate data in DataFrames. A DataFrame in its simplest description, is a table, with rows and columns. When we want to handle an excel file in python, we need to import it into a pandas DataFrame. The following code uses the pandas function read_excel to read an excel at a specified location, use the first row in the excel sheet as the header, and the first column as the index, and assign it to a DataFrame named samples.

In [ ]:
samples = pd.read_excel('C:\Apache24\htdocs\geochem_samples.xlsx', header=[0], index_col=0)

Now let's do some simple exploratory data analysis to see what we are working with. We have a few things we can easily look at with simple commands.

The <b>head</b> method will print the first few lines of the DataFrame, with the parameter being passed as the number of lines to print (5 is the default).
The <b>tail</b> method does the same for the last lines.

In [ ]:
print(samples.head(3))
print(samples.tail(3))

Some other commands include <b>info</b>, which gives information on how many rows are in the DataFrame, the number of columns, null values, and the datatypes of each column. The <b>describe</b> method calculates and displays some basic statistics of the samples(count, mean, standard deviation, minimum, maximum, and 25, 50, and 75 percentiles. Finally the <b>columns</b> attribute gives an Index of all the column names. Looking at your data in this way is important, as it gives you a good idea of any clean up you may have to do.

In [ ]:
print(samples.info())

In [ ]:
print(samples.describe())

In [ ]:
print(samples.columns) # Note: columns is an attribute of the samples DataFrame, so as opposed to the previous methods, does not need paranthesis.

Let's print out the whole DataFrame to see what we are working with.

In [ ]:
print(samples)

So we have 24 samples. One feature is Area, one is Fragment Type, and 5 trace geochemical elements (Ba, Cr, Ni, Sr, V) all in ppm. So let's ask some questions. Say we want to know the average(mean) for the samples in each area. We can use the <b>groupby</b> method on samples, chained with the <b>mean</b> method. We also add the paramater <b>as_index=True</b> so the new DataFrame uses whatever we groupby, as the index for the new DataFrame.

In [ ]:
samples_loc_summary = samples.groupby('Area', as_index=True).mean()
print(samples_loc_summary)

We can also groupby multiple features by passing a list into the first argument.

In [ ]:
groups = ['Area', 'Fragment Type']
samples_frag_area_summary = samples.groupby(groups, as_index=True).mean()
#samples_frag_area_summary = samples.groupby(['Area', 'Fragment Type'], as_index=True).mean()   # Alternatively you could do this.
print(samples_frag_area_summary)

Now say we just want to see the M(Mafic) samples, and we also want to see it ordered by the highest Barium values to the lowest. In this case we filter out 'M' samples, by nesting a boolean statement as in the code below. And use the <b>sort_values</b> method, with the ascending=False parameter to order the DataFrame as required.

In [ ]:
mafic_samples = samples[samples['Fragment Type'] == 'M']
ordered_mafic_samples = mafic_samples.sort_values('Ba  ppm', ascending=False)
print(ordered_mafic_samples.head()) # Let's just see the top 5 values.

Ok. We've run through a lot of stuff really quickly. Some people may have already asked the question "But what do I do when my Excel file has multiple sheets?". In this case let's use the same file and instead of using the function <b>read_excel</b>, we will use <b>ExcelFile</b>. This pandas function reads the entire excel file into an Excel Object(note: Not a DataFrame, we will get to that). Let's do that a print it out to see what happens.

In [ ]:
samples_workbook = pd.ExcelFile('C:\Apache24\htdocs\geochem_samples.xlsx')
print(samples_workbook)

Interesting, but not particularily useful to use yet. This new Excel Object has an attribute <b>sheet_names<b>, which you can probably guess at what this is. Let's try printing it out.

In [ ]:
print(samples_workbook.sheet_names)

Awesome! Now we will use the method <b>parse</b> to assign each sheet into it's own DataFrame (and let's print out the first 3 lines to get an idea of what we are looking at).

In [ ]:
geochem = samples_workbook.parse('Geochem') # Pass the string Geochem in as a parameter in this case.
locations = samples_workbook.parse('Locations') # Same as above, but with the Locations sheet.
print(geochem.head(3))
print(locations.head(3))

Being able to reorganize your DataFrames by selecting only certain columns, or dropping some columns is a good skill to have. So let's do that! Say for our geochem data, we are only interested in nickel.  We can assign only the Sample Name, and Ni ppm columns to a new DataFrame as follows:

In [ ]:
geochem_Ni = geochem[['Sample Name', 'Ni  ppm']]
print(geochem_Ni.head())

We can also drop columns using the <b>drop</b> method. Let's drop the 'Area' and 'ELEV' columns from the locations DataFrame. In this case we pass the axis=1 parameter to specify we are dropping a column (axis=0 would drop rows).

In [ ]:
samples_UTM = locations.drop(['ELEV', 'Area'], axis=1)
print(samples_UTM.head())

Now we may want to join the locations, with the nickel values. To do this we use the <b>merge</b> method, and in this case we will left join. We need to apply merge to a DataFrame(geochem_Ni), and pass 3 parameters, the DataFrame we want to merge onto (samples_UTM), the column we want to merge on (on = 'Sample Name'), and how we want to merge the 2 (how = 'left'). Let's also use <b>set_index</b> method to assign the Sample Name as the index. We will also set the parameter <b>inplace=True</b> to do this modification on the object itself.

In [ ]:
samples_Ni_loc = geochem_Ni.merge(samples_UTM, on='Sample Name', how='left')
samples_Ni_loc.set_index('Sample Name', inplace=True)
print(samples_Ni_loc.head())

And that's the basics! As a bonus...  let's do some visualizations! For this we need some new packages. We will use seaborn, and matplotlib. So let's import those.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

I'm going to run through this a bit quickly, but I do plan on doing another notebook focuses primarly on visualization, so no worries! In this case we are going to use the seaborn method<b>set_style</b> to give our plot a nice white grid. For no good reason at all, we will use a box plot to view our Nickel values by Area. See the code below on how to do this.

In [ ]:
sns.set_style('whitegrid')                                 # Set the barplot style
sns.barplot(x='Area', y='Ni  ppm', data=samples, ci=None)  # Create a seaborn barlot, x being area, y is Ni ppm, and no confidence interval is plotted.
plt.title('Nickel values by Area')                         # Title the plot
plt.xlabel('Fragment')                                     # Label the x axis
plt.ylabel('Nickel(ppm)')                                  # Label the y axis
sns.despine()                                              # This removes the outside border of the plot, and is purely aesthetic
plt.show()                                                 # Without this, your plot will exist, but won't actually print on screen.  Use this to view.